In [1]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [2]:
#路径
path='C:\Users\Wvv\Datacastle_subsidy'

#导入library_train数据（图书馆门禁）
library_test_name=['id','lib_dor_num','lib_date']
library_test=pd.read_table(path + '/final_test/library_final_test.txt',sep=',',header=None,names=library_test_name)

#删除重复行并浏览前几行数据
library_test=library_test.drop_duplicates()
library_test.head()

,id,lib_dor_num,lib_date
0,6308,7,2013/09/01 08:42:43
1,20745,5,2013/09/01 08:42:49
2,8152,6,2013/09/01 08:43:42
3,28693,5,2013/09/01 08:45:32
4,5466,6,2013/09/01 08:45:35


In [2]:
#路径
path='C:\Users\Wvv\Datacastle_subsidy'

#导入library_train数据（图书馆门禁）
library_test_name=['id','lib_dor_num','lib_date']
library_test=pd.read_table(path + '/test/library_test.txt',sep=',',header=None,names=library_test_name)

#删除重复行并浏览前几行数据
library_test=library_test.drop_duplicates()
library_test.head()

,id,lib_dor_num,lib_date
0,3094,6,2013/09/01 08:42:32
1,9812,5,2013/09/01 08:42:46
2,1972,7,2013/09/01 08:42:51
3,5508,7,2013/09/01 08:43:22
4,7060,6,2013/09/01 08:45:17


In [3]:
library_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048633 entries, 0 to 1055195
Data columns (total 3 columns):
id             1048633 non-null int64
lib_dor_num    1048633 non-null object
lib_date       1048633 non-null object
dtypes: int64(1), object(2)
memory usage: 32.0+ MB


In [3]:
#定义time转化函数

from datetime import datetime
def time_transform(x):
    return datetime.strptime(x,'%Y/%m/%d %H:%M:%S')

In [4]:
#转换字符串型的time成时间戳格式
library_test['lib_date']=library_test['lib_date'].apply(time_transform)

In [5]:
#时间格式转换

library_test['date']=library_test['lib_date'].apply(lambda x:x.strftime('%m-%d-%Y'))
library_test['year']=library_test['lib_date'].apply(lambda x:x.strftime('%Y'))
library_test['month']=library_test['lib_date'].apply(lambda x:x.strftime('%m'))
library_test['hour']=library_test['lib_date'].apply(lambda x:x.strftime('%H'))
library_test['weekday']=library_test['lib_date'].apply(lambda x:x.strftime('%A'))

In [6]:
#获得图书馆开放时段

library_hour=library_test.groupby('hour')['hour'].count()
library_hour.index

Index([u'00', u'01', u'06', u'07', u'08', u'09', u'10', u'11', u'12', u'13',
       u'14', u'15', u'16', u'17', u'18', u'19', u'20', u'21', u'22', u'23'],
      dtype='object', name=u'hour')

In [8]:
#hour(每个时段)

a=['00','01','06','23']
def hour_transform(hour):
    if hour in a:
        return 'o'
    else:
        return hour
    
            
library_test['hour']=library_test['hour'].apply(hour_transform)

#对hour进行one_hot编码得到hour新表
hour_dummies=pd.get_dummies(library_test['hour'])
hour_dummies.columns = ['lib_hour_07','lib_hour_08','lib_hour_09','lib_hour_10','lib_hour_11','lib_hour_12','lib_hour_13','lib_hour_14',
                       'lib_hour_15','lib_hour_16','lib_hour_17','lib_hour_18','lib_hour_19','lib_hour_20','lib_hour_21','lib_hour_22','lib_hour_other']

#连接两表
library_test = library_test.join(hour_dummies)

In [9]:
#hour(早，中，晚)

a=['o','07','08','09','10','11']
b=['12','13','14','15','16','17']
#c=['18','19','20','21','22']
def hour_transform(hour_1):
    if hour_1 in a:
        return 'a'
    elif hour_1 in b:
        return 'b'
    else:
        return 'c'
    
            
library_test['hour']=library_test['hour'].apply(hour_transform)

#对hour进行one_hot编码得到hour新表
hour_dummies_1=pd.get_dummies(library_test['hour'])
hour_dummies_1.columns = ['lib_morning','lib_afternoon','lib_evening']

#连接两表
library_test = library_test.join(hour_dummies_1)

In [10]:
#weekday

#分为周末与非周末
a=['Monday','Tuesday','Wednesday','Thursday','Friday']
#b=['Saturday','Sunday']
def week_transform(weekday):
    if weekday in a:
        return 'wd'
    else:
        return 'vd'
    
library_test['weekday']=library_test['weekday'].apply(week_transform)

#对weekday进行one_hot编码得到weekday新表

weekday_dummies = pd.get_dummies(library_test['weekday'])
weekday_dummies.columns=['lib_resting_day','lib_working_day']
library_test=library_test.join(weekday_dummies)

In [11]:
#month(每个月)

#对month进行one_hot编码得到month新表

month_dummies = pd.get_dummies(library_test['month'])
month_dummies.columns=['lib_month_01','lib_month_02','lib_month_03','lib_month_04','lib_month_05','lib_month_06','lib_month_07'
                      ,'lib_month_08','lib_month_09','lib_month_10','lib_month_11','lib_month_12']
library_test=library_test.join(month_dummies)

In [12]:
#month

#分为四个季节

a=['03','04','05']#春季
b=['06','07','08']#夏季
c=['09','10','11']#秋季
#d=['12','01','02']#冬季
def month_transform(month_1):
    if month_1 in a:
        return 'a'
    elif month_1 in b:
        return 'b'
    elif month_1 in c:
        return 'c'
    else:
        return 'd'
    
library_test['month']=library_test['month'].apply(month_transform)

#对month进行one_hot编码得到month新表

month_dummies_1 = pd.get_dummies(library_test['month'])
month_dummies_1.columns=['lib_spring','lib_summer','lib_autumn','lib_winter']
library_test=library_test.join(month_dummies_1)

In [13]:
library_test.head()

,id,lib_dor_num,lib_date,date,year,month,hour,weekday,lib_hour_07,lib_hour_08,...,lib_month_07,lib_month_08,lib_month_09,lib_month_10,lib_month_11,lib_month_12,lib_spring,lib_summer,lib_autumn,lib_winter
0,3094,6,2013-09-01 08:42:32,09-01-2013,2013,c,a,vd,0,1,...,0,0,1,0,0,0,0,0,1,0
1,9812,5,2013-09-01 08:42:46,09-01-2013,2013,c,a,vd,0,1,...,0,0,1,0,0,0,0,0,1,0
2,1972,7,2013-09-01 08:42:51,09-01-2013,2013,c,a,vd,0,1,...,0,0,1,0,0,0,0,0,1,0
3,5508,7,2013-09-01 08:43:22,09-01-2013,2013,c,a,vd,0,1,...,0,0,1,0,0,0,0,0,1,0
4,7060,6,2013-09-01 08:45:17,09-01-2013,2013,c,a,vd,0,1,...,0,0,1,0,0,0,0,0,1,0


In [14]:
#导出数据

library_test.to_csv('lib_input/library_test.csv',index=False)

In [14]:
#导出数据

library_test.to_csv('lib_input/library_test_2.csv',index=False)